<a href="https://colab.research.google.com/github/Aditya-1301/ToonCrafter_Experiment/blob/main/init_tc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------------------------------------------------------------------------------
# Cell 1: Check GPU and Initial Setup
# ------------------------------------------------------------------------------------------
!nvcc --version
!python --version

# Ensure we're in the main content directory
import os
os.chdir('/content/')
print(f"Current working directory: {os.getcwd()}")

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Python 3.11.12
Current working directory: /content


In [ ]:
# ------------------------------------------------------------------------------------------
# Cell 2: Clone the ToonCrafter Repository
# ------------------------------------------------------------------------------------------
!git clone https://github.com/Doubiiu/ToonCrafter.git
os.chdir('/content/ToonCrafter')
!pwd

Cloning into 'ToonCrafter'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 177 (delta 20), reused 6 (delta 6), pack-reused 134 (from 1)
Receiving objects: 100% (177/177), 68.95 MiB | 11.81 MiB/s, done.
Resolving deltas: 100% (25/25), done.
/content/ToonCrafter


In [ ]:
# ------------------------------------------------------------------------------------------
# Cell 3: Install Dependencies (Revised Aggressive Approach)
# ------------------------------------------------------------------------------------------

# Step 0: Uninstall potentially conflicting packages and clear pip cache
print("Uninstalling potentially conflicting packages to ensure a cleaner state...")
!pip uninstall -y tensorflow tensorboard tb-nightly tf-nightly Keras-Preprocessing Keras-Applications Keras tensorflow-datasets tensorflow-hub tensorflow-io tensorflow-metal tensorflow-recommenders tensorflow-text tensorflow_decision_forests tensorflow_ranking tensorflow_transform google-colab jax jaxlib pandas numpy protobuf transformers sentence-transformers torchmetrics pytorch-lightning albumentations
!pip cache purge
print("Uninstall and cache purge completed.")

# Step 1: Install specific PyTorch, torchvision, torchaudio (as per README, for CUDA 11.8)
print("\nInstalling PyTorch, torchvision, and torchaudio for CUDA 11.8...")
# Using --force-reinstall to be absolutely sure.
!pip install --no-cache-dir --force-reinstall torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
print("PyTorch, torchvision, and torchaudio installation attempted.")

# Verify PyTorch version and CUDA availability immediately and strictly
import torch
print(f"PyTorch version: {torch.__version__}")
# Check if the main version string starts with 2.0.1. Some pip versions might add +cu118 etc.
if not str(torch.__version__).startswith("2.0.1"):
    print(f"FATAL ERROR: Expected PyTorch 2.0.1, but got {torch.__version__}. Cannot proceed.")
    # You might need to stop here if this happens.
    # Consider restarting the Colab Runtime and trying again if this fails.
else:
    print("PyTorch version 2.0.1 confirmed.")
print(f"CUDA available for PyTorch: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version PyTorch is built with: {torch.version.cuda}")
    print(f"Current CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("ERROR: CUDA is NOT available for PyTorch. This will prevent GPU usage.")

try:
    import torchvision
    print(f"Torchvision version: {torchvision.__version__}")
    if hasattr(torchvision.ops, 'nms'):
        print("torchvision.ops.nms symbol is available.")
    else:
        print("WARNING: torchvision.ops.nms symbol is NOT available (potential runtime error).")
except Exception as e:
    print(f"An error occurred during torchvision check: {e}")

# Step 2: Install specific xformers version
print("\nAttempting to install xformers==0.0.20...")
!pip install --no-cache-dir xformers==0.0.20
print("xformers==0.0.20 installation attempted.")

try:
    import xformers
    import xformers.ops
    print(f"xformers version: {xformers.__version__} successfully imported.")
except Exception as e:
    print(f"ERROR: Failed to import xformers==0.0.20: {e}. ToonCrafter may not work.")

# Step 3: Install packages from ToonCrafter's requirements.txt
print("\nInstalling packages from requirements.txt (expect potential conflict warnings)...")
# This will install pandas==2.0.0, transformers==4.25.1, albumentations==0.5.2 etc.
# It will also bring in torchmetrics -> (transformers) -> tensorflow -> jax
!pip install -r requirements.txt
print("requirements.txt packages installation attempted.")

# Let's see what versions requirements.txt actually installed for problematic libs
print("\n--- Versions after requirements.txt ---")
!pip show numpy pandas protobuf transformers torchmetrics pytorch-lightning albumentations tensorflow jax jaxlib

# Step 4: Targeted upgrades/reinstalls to resolve known critical conflicts
# NumPy: Upgrade to satisfy TensorFlow (>=1.26) and JAX (>=1.25)
print("\nUpgrading NumPy to ~1.26.0 to satisfy TensorFlow and JAX...")
!pip install -U "numpy>=1.26.0,<1.27.0" # Target 1.26.x, e.g. 1.26.4
print("NumPy upgrade attempted.")

# Protobuf: Upgrade to satisfy tensorflow-metadata and other TF components
print("\nUpgrading Protobuf...")
!pip install -U "protobuf>=4.25.2,<5.0.0" # Common requirement range
print("Protobuf upgrade attempted.")

# Re-pinning key ToonCrafter dependencies from its requirements.txt, in case upgrades above affected them.
# This is mostly for accelerate, diffusers, einops, as requirements.txt pins them.
print("\nRe-installing specific versions of accelerate, diffusers, einops from requirements.txt...")
!pip install accelerate==0.25.0 diffusers==0.25.0 einops==0.7.0
print("accelerate, diffusers, einops re-installation attempted.")

# Pandas: ToonCrafter wants pandas==2.0.0. google-colab wants 2.2.2.
# We will stick with 2.0.0 from requirements.txt. The google-colab warning is usually benign.
# Ensure pandas is 2.0.0
print("\nEnsuring pandas is at version 2.0.0 as per requirements.txt...")
!pip install pandas==2.0.0
print("Pandas version check/reinstall attempted.")

# Albumentations: ToonCrafter wants 0.5.2.
print("\nEnsuring albumentations is at version 0.5.2 as per requirements.txt...")
!pip install albumentations==0.5.2
print("Albumentations version check/reinstall attempted.")


# --- Final versions of key packages ---
print("\n--- Final versions of key packages after all steps ---")
!pip show torch torchvision torchaudio xformers numpy pandas protobuf transformers torchmetrics pytorch-lightning accelerate diffusers einops albumentations tensorflow jax jaxlib

# --- Final Sanity Checks (from previous response, good to keep) ---
print("\n--- Final Sanity Dependency Check ---")
try:
    import torch
    print(f"Final PyTorch version: {torch.__version__}")
    print(f"Final CUDA available for PyTorch: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"Final CUDA version PyTorch is built with: {torch.version.cuda}")
except Exception as e: print(f"PyTorch check error: {e}")

try:
    import torchvision
    print(f"Final Torchvision version: {torchvision.__version__}")
    if hasattr(torchvision.ops, 'nms'): print("Final torchvision.ops.nms: Available")
    else: print("Final torchvision.ops.nms: NOT Available (Potential Issue!)")
except Exception as e: print(f"Torchvision check error: {e}")

try:
    import xformers
    print(f"Final xformers version: {xformers.__version__}")
except Exception as e: print(f"xformers check error: {e}")

try:
    import numpy
    print(f"Final NumPy version: {numpy.__version__}")
    print(f"Final NumPy has dtypes: {hasattr(numpy, 'dtypes')}")
    if not hasattr(numpy, 'dtypes'): print("CRITICAL WARNING: Final NumPy still lacks 'dtypes' attribute!")
except Exception as e: print(f"NumPy check error: {e}")

try:
    # Check if JAX can be imported and sees a compatible NumPy
    import jax
    import jax.numpy as jnp
    print(f"Final JAX version: {jax.__version__}")
    # A simple JAX operation that might use NumPy internals
    try:
        _ = jnp.array([1,2,3]) + 1
        print("Basic JAX operation successful.")
    except Exception as e_jax_op:
        print(f"Error during basic JAX operation: {e_jax_op}")
except Exception as e: print(f"JAX check error: {e}")

print("-----------------------------------")

Uninstalling potentially conflicting packages to ensure a cleaner state...
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: tensorboard 2.18.0
Uninstalling tensorboard-2.18.0:
  Successfully uninstalled tensorboard-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: tensorflow-datasets 4.9.8
Uninstalling tensorflow-datasets-4.9.8:
  Successfully uninstalled tensorflow-datasets-4.9.8
Found existing installation: tensorflow-hub 0.16.1
Uninstalling tensorflow-hub-0.16.1:
  Successfully uninstalled tensorflow-hub-0.16.1
Found existing installation: tensorflow-text 2.18.1
Uninstalling tensorflow-text-2.18.1:
  Successfully uninstalled tensorflow-text-2.18.1
Found existing installation: tensorflow_decision_forests 1.11.0
Uninstalling tensorflow_decision_forests-1.11.0:
  Successfully uninstalled tenso

PyTorch, torchvision, and torchaudio installation attempted.
PyTorch version: 2.0.1+cu118
PyTorch version 2.0.1 confirmed.
CUDA available for PyTorch: False
ERROR: CUDA is NOT available for PyTorch. This will prevent GPU usage.



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Torchvision version: 0.15.2+cu118
torchvision.ops.nms symbol is available.

Attempting to install xformers==0.0.20...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 137.0 MB/s eta 0:00:00
xformers==0.0.20 installation attempted.
xformers version: 0.0.20 successfully imported.

Installing packages from requirements.txt (expect potential conflict warnings)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You m

requirements.txt packages installation attempted.

--- Versions after requirements.txt ---
Name: numpy
Version: 1.24.2
Summary: Fundamental package for array computing in Python
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: accelerate, albucore, ale-py, arviz, astropy, autograd, bigframes, blis, blosc2, bokeh, Bottleneck, bqplot, chex, clarabel, cmdstanpy, contourpy, cudf-cu12, cufflinks, cuml-cu12, cupy-cuda12x, cuvs-cu12, cvxpy, cyipopt, dask-cuda, dask-cudf-cu12, datascience, datasets, db-dtypes, decord, diffusers, dm-tree, dopamine_rl, flax, folium, geemap, geopandas, gradio, gym, gymnasium, h5py, hdbscan, highspy, holoviews, hyperopt, imageio, imbalanced-learn, keras-hub, libpysal, librosa, lightgbm, matplotlib, matplotlib-venn, missingno, mizani, ml-dtypes, mlxtend, moviepy, music21, nibabel, numba, numexpr, nx-cugraph-cu12, opencv-contrib-python, op

Albumentations version check/reinstall attempted.

--- Final versions of key packages after all steps ---
Name: torch
Version: 2.0.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, jinja2, networkx, nvidia-cublas-cu11, nvidia-cuda-cupti-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, nvidia-cufft-cu11, nvidia-curand-cu11, nvidia-cusolver-cu11, nvidia-cusparse-cu11, nvidia-nccl-cu11, nvidia-nvtx-cu11, sympy, triton, typing-extensions
Required-by: accelerate, fastai, kornia, open-clip-torch, peft, pytorch-lightning, timm, torchaudio, torchdata, torchmetrics, torchvision, triton, xformers
---
Name: torchvision
Version: 0.15.1
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Cor

In [ ]:
# ------------------------------------------------------------------------------------------
# Cell 4: Download Pretrained Checkpoints
# ------------------------------------------------------------------------------------------
# Create the checkpoints directory if it doesn't exist
%cd /content/ToonCrafter/
!mkdir -p checkpoints

# Download ToonCrafter_512.ckpt (model_default)
print("\nDownloading ToonCrafter_512.ckpt...")
!wget -P checkpoints/ https://huggingface.co/Doubiiu/ToonCrafter/blob/main/model.ckpt

# # Download ToonCrafter_MVP_512.ckpt (model_mv)
# print("\nDownloading ToonCrafter_MVP_512.ckpt...")
# !wget -P checkpoints/https://huggingface.co/Doubiiu/ToonCrafter/blob/main/model.ckpt

# Verify downloads
print("\nContents of checkpoints directory:")
!ls -lh checkpoints/

/content

--2025-05-20 21:46:27--  https://huggingface.co/Doubiiu/ToonCrafter/blob/main/model.ckpt
Resolving huggingface.co (huggingface.co)... 65.8.243.90, 65.8.243.92, 65.8.243.46, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51336 (50K) [text/html]
Saving to: ‘checkpoints/model.ckpt’

model.ckpt          100%[===================>]  50.13K  --.-KB/s    in 0.01s   

2025-05-20 21:46:27 (4.21 MB/s) - ‘checkpoints/model.ckpt’ saved [51336/51336]


Contents of checkpoints directory:
total 52K
-rw-r--r-- 1 root root 51K May 20 21:46 model.ckpt


In [ ]:
# ------------------------------------------------------------------------------------------
# Cell 5: Run Inference with Gradio Web UI
# ------------------------------------------------------------------------------------------
# Option 1: Run with the default model (ToonCrafter_512.ckpt)
# The Gradio interface will provide a public URL if running in Colab.
print("\nStarting Gradio UI with default model (ToonCrafter_512)...")
# Ensure you are in the ToonCrafter directory
import os
os.chdir('/content/ToonCrafter')
# !python gradio_inference.py --model_name model_default
!python gradio_app.py

# Option 2: Run with the MVP model (ToonCrafter_MVP_512.ckpt)
# Comment out the above command and uncomment the one below if you want to try this model instead.
# print("\nStarting Gradio UI with MVP model (ToonCrafter_MVP_512)...")
# os.chdir('/content/ToonCrafter')
# !python gradio_inference.py --model_name model_mv


Starting Gradio UI with default model (ToonCrafter_512)...
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
model.ckpt: 100% 10.5G/10.5G [01:58<00:00, 88.6MB/s]
AE working on z of shape (1, 4, 32, 32) = 4096 dimensions.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
open_clip_pytorch_model.bin: 100% 3.94G/3.94G [06:23<00:00, 10.3MB/s]
^C


In [ ]:
import numpy as np
np.__version__

'2.0.2'

In [ ]:
!python gradio_app.py

AE working on z of shape (1, 4, 32, 32) = 4096 dimensions.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
^C


In [1]:
%cd /content/ToonCrafter/
!sh scripts/run.sh

[Errno 2] No such file or directory: '/content/ToonCrafter/'
/content
sh: 0: cannot open scripts/run.sh: No such file
